<a href="https://colab.research.google.com/github/sumit778285/Visual-Question-Answering_BLIP/blob/main/visual%20question%20answering%20using%20BLIP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
!pip install transformers torchvision accelerate


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 67.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 35.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 40.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 29.5 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [12]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [13]:
import pandas as pd
import ast

# Path to your CSV file (adjust if needed)
csv_path = '/content/drive/MyDrive/VQA/dataset.csv'
df = pd.read_csv(csv_path)

# Convert stringified list to list
df['images'] = df['images'].apply(ast.literal_eval)
df['image_path'] = df['images'].apply(lambda x: x[0] if len(x) > 0 else None)

# Drop missing or "None" answers
df = df.dropna(subset=['image_path', 'question', 'answer'])
df = df[df['answer'].str.lower() != 'none'].reset_index(drop=True)

# Show sample
df[['image_path', 'question', 'answer']].head()


,image_path,question,answer
0,images/val/f2e4286e94457b8605069190e29f955a/0.jpg,"Is there a traffic light? If yes, what color i...","Yes, green."
1,images/val/f2e4286e94457b8605069190e29f955a/0.jpg,"Is there a traffic light? If yes, what color i...","Yes, a temporary traffic light. It is showing ..."
2,images/val/f2e4286e94457b8605069190e29f955a/0.jpg,How many pedestrians can be seen?,None.
3,images/val/f2e4286e94457b8605069190e29f955a/0.jpg,How many pedestrians can be seen?,No pedestrian can be seen.
4,images/val/f2e4286e94457b8605069190e29f955a/0.jpg,What is the current action and its justificati...,"The car starts and moves right, because the ve..."


In [14]:
import torch
from torch.utils.data import Dataset
from PIL import Image
from transformers import BlipProcessor
import os

class VQADataset(Dataset):
    def __init__(self, dataframe, image_root, processor):
        self.df = dataframe
        self.image_root = image_root
        self.processor = processor

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        item = self.df.iloc[idx]

        # Full image path preserving subdirectories
        relative_img_path = item['image_path']
        img_path = os.path.join(self.image_root, os.path.relpath(relative_img_path, 'images/val'))

        question = item['question']
        answer = item['answer']

        # Load and process image
        image = Image.open(img_path).convert('RGB')

        # Processor handles image + question
        inputs = self.processor(
            images=image,
        text=question,
        return_tensors="pt",
        padding='max_length',
        truncation=True,
        max_length=128
        )

        # Tokenize answer for supervision
        labels_data = self.processor.tokenizer(
            answer,
            return_tensors="pt",
            padding='max_length',
            truncation=True,
            max_length=128
        )

        inputs["labels"] = labels_data["input_ids"].squeeze(0)
        inputs["decoder_attention_mask"] = labels_data["attention_mask"].squeeze(0)

        return {k: v.squeeze(0) for k, v in inputs.items()}




In [15]:
from transformers import BlipProcessor, BlipForQuestionAnswering

# Load processor and model
processor = BlipProcessor.from_pretrained("Salesforce/blip-vqa-base")
model = BlipForQuestionAnswering.from_pretrained("Salesforce/blip-vqa-base")

# Set model to CPU
device = torch.device("cpu")
model.to(device)


Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


BlipForQuestionAnswering(
  (vision_model): BlipVisionModel(
    (embeddings): BlipVisionEmbeddings(
      (patch_embedding): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
    )
    (encoder): BlipEncoder(
      (layers): ModuleList(
        (0-11): 12 x BlipEncoderLayer(
          (self_attn): BlipAttention(
            (dropout): Dropout(p=0.0, inplace=False)
            (qkv): Linear(in_features=768, out_features=2304, bias=True)
            (projection): Linear(in_features=768, out_features=768, bias=True)
          )
          (layer_norm1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (mlp): BlipMLP(
            (activation_fn): GELUActivation()
            (fc1): Linear(in_features=768, out_features=3072, bias=True)
            (fc2): Linear(in_features=3072, out_features=768, bias=True)
          )
          (layer_norm2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        )
      )
    )
    (post_layernorm): LayerNorm((768,), eps=1e-05, e

In [16]:
from torch.utils.data import DataLoader

# Set your image root folder (adjust path as needed)
image_root = '/content/drive/MyDrive/VQA/images/val'

# Dataset and DataLoader
dataset = VQADataset(df, image_root, processor)
dataloader = DataLoader(dataset, batch_size=2, shuffle=True)


In [17]:
from torch.optim import AdamW
from tqdm import tqdm

optimizer = AdamW(model.parameters(), lr=5e-5)
model.train()

for epoch in range(2):  # You can increase epochs later
    loop = tqdm(dataloader, leave=True)
    for batch in loop:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        loop.set_description(f"Epoch {epoch}")
        loop.set_postfix(loss=loss.item())


Epoch 1: 100%|██████████| 20/20 [07:04<00:00, 21.24s/it, loss=5.87]


In [18]:
model.save_pretrained('/content/blip-vqa-cpu')
processor.save_pretrained('/content/blip-vqa-cpu')


[]

In [19]:
def predict(image_path, question):
    model.eval()
    image = Image.open(image_path).convert("RGB")
    inputs = processor(image, question, return_tensors="pt").to(device)
    out = model.generate(**inputs)
    return processor.decode(out[0], skip_special_tokens=True)

# Example:
predict('/content/drive/MyDrive/VQA/images/val/f2e4286e94457b8605069190e29f955a/0.jpg', "What is the color of the traffic light?")


'green'

In [20]:
!pip install nltk
!pip install git+https://github.com/salaniz/pycocoevalcap


  Cloning https://github.com/salaniz/pycocoevalcap to /tmp/pip-req-build-sakl0m1v
  Running command git clone --filter=blob:none --quiet https://github.com/salaniz/pycocoevalcap /tmp/pip-req-build-sakl0m1v
  Resolved https://github.com/salaniz/pycocoevalcap to commit a24f74c408c918f1f4ec34e9514bc8a76ce41ffd
  Preparing metadata (setup.py) ... done


In [21]:
# Step 1: Clean uninstall
!pip uninstall -y nltk

# Step 2: Reinstall fresh
!pip install nltk

# Step 3: Download 'punkt' tokenizer
import nltk
nltk.download('punkt')


Found existing installation: nltk 3.9.1
Uninstalling nltk-3.9.1:
  Successfully uninstalled nltk-3.9.1
  Using cached nltk-3.9.1-py3-none-any.whl.metadata (2.9 kB)
Using cached nltk-3.9.1-py3-none-any.whl (1.5 MB)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [23]:
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt_tab') # Download the required resource


[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [24]:
from tqdm import tqdm
import os
import nltk
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from pycocoevalcap.cider.cider import Cider
from nltk.tokenize import word_tokenize

references = []
hypotheses = []

cider_refs = {}
cider_hyps = {}

subset_df = df.iloc[:40].reset_index(drop=True)

for i, row in tqdm(subset_df.iterrows(), total=len(subset_df)):
    rel_path = os.path.relpath(row['image_path'], 'images/val')
    image_path = os.path.join('/content/drive/MyDrive/VQA/images/val', rel_path)

    question = row['question']
    gt = row['answer']
    pred = predict(image_path, question)

    # BLEU preparation
    references.append([nltk.word_tokenize(gt.lower())])
    hypotheses.append(nltk.word_tokenize(pred.lower()))

    # CIDEr preparation
    qid = f"q{i}"
    cider_refs[qid] = [gt]
    cider_hyps[qid] = [pred]

100%|██████████| 39/39 [02:42<00:00,  4.17s/it]


In [25]:
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction

smooth = SmoothingFunction().method4

bleu_scores = [
    sentence_bleu(ref, hyp, smoothing_function=smooth)
    for ref, hyp in zip(references, hypotheses)
]

average_bleu = sum(bleu_scores) / len(bleu_scores)
print(f"\n🔵 Average BLEU score over 40 samples: {average_bleu:.4f}")



🔵 Average BLEU score over 40 samples: 0.0884


In [31]:
cider = Cider()
cider_score, _ = cider.compute_score(cider_refs, cider_hyps)

print(f"🟢 Average CIDEr score over 40 samples: {cider_score:.4f}")


🟢 Average CIDEr score over 40 samples: 0.2166


In [32]:
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from pycocoevalcap.cider.cider import Cider
from nltk.tokenize import word_tokenize
from tqdm import tqdm
import os

smooth = SmoothingFunction().method4
cider_scorer = Cider()

# To store results
results = []

for i, row in tqdm(subset_df.iterrows(), total=len(subset_df)):
    rel_path = os.path.relpath(row['image_path'], 'images/val')
    image_path = os.path.join('/content/drive/MyDrive/VQA/images/val', rel_path)

    question = row['question']
    ground_truth = row['answer']
    prediction = predict(image_path, question)

    # --- BLEU ---
    ref_tokens = [word_tokenize(ground_truth.lower())]
    pred_tokens = word_tokenize(prediction.lower())
    bleu = sentence_bleu(ref_tokens, pred_tokens, smoothing_function=smooth)

    # --- CIDEr ---
    # Wrap strings in list-of-list structure
    ref_dict = {f"q{i}": [ground_truth.strip()]}
    pred_dict = {f"q{i}": [prediction.strip()]}

    try:
        cider, _ = cider_scorer.compute_score(ref_dict, pred_dict)
    except Exception as e:
        print(f"Error on sample {i}: {e}")
        cider = 0.0


    # Store the result
    results.append({
        'question': question,
        'ground_truth': ground_truth,
        'prediction': prediction,
        'bleu': round(bleu, 4),
        'cider': round(cider, 4)
    })


100%|██████████| 39/39 [02:03<00:00,  3.16s/it]


In [33]:
import pandas as pd

results_df = pd.DataFrame(results)
pd.set_option("display.max_colwidth", None)

# Display the first 5 results
#results_df.head()

# Show full content (no truncation)
pd.set_option("display.max_colwidth", None)

# Show all rows
pd.set_option("display.max_rows", None)

# Display full table
results_df


,question,ground_truth,prediction,bleu,cider
0,"Is there a traffic light? If yes, what color is displayed?","Yes, green.",yes,0.0498,0.0
1,"Is there a traffic light? If yes, what color is displayed?","Yes, a temporary traffic light. It is showing green.",yes,0.0000,0.0
2,How many pedestrians can be seen?,None.,there is one pedestrian crossing the road in front of the traffic light,0.0000,0.0
3,How many pedestrians can be seen?,No pedestrian can be seen.,there is one pedestrian crossing the road in front of the traffic light,0.0187,0.0
4,"What is the current action and its justification? Answer in the form ""action, justification"".","The car starts and moves right, because the vehicle in front is pulling away at the green traffic light and then it needs to go around the construction sign on the left of the road.",the car is in front of the traffic light,0.0058,0.0
5,"What is the current action and its justification? Answer in the form ""action, justification"".","Starting from stop, as the traffic light turns green and the van in front accelerates.",the car is in front of the traffic light,0.0782,0.0
6,Are there any pedestrian crossing ahead?,There is no pedestrian crossing visible ahead.,yes,0.0000,0.0
7,Are there any pedestrian crossing ahead?,No.,yes,0.0000,0.0
8,How many motorcyclists can be seen?,There is one motorcyclist behind the truck on the opposite lane.,there is one pedestrian crossing the road in front of the traffic light,0.1106,0.0
9,How many motorcyclists can be seen?,1,there is one pedestrian crossing the road in front of the traffic light,0.0000,0.0
